In [65]:
import pandas as pd

In [66]:
from trading_bot.agent import Agent
model_name = 'model_double-dqn_GOOG_50'
test_stock = 'data/BTCUSDT_TESTING.csv'
window_size = 10
debug = True
agent = Agent(window_size, pretrained=True, model_name=model_name)

In [67]:
df = pd.read_csv(test_stock)
df = df[['Date', 'Close']]
df = df.rename(columns={'Close': 'actual', 'Date': 'date'})
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates
df

,date,actual
0,2019-08-23,10389.55
1,2019-08-24,10134.35
2,2019-08-25,10142.69
3,2019-08-26,10372.25
4,2019-08-27,10185.05
...,...,...
248,2020-04-27,7774.62
249,2020-04-28,7738.98
250,2020-04-29,8778.57
251,2020-04-30,8620.00


In [68]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-05-26 15:38:24 darkdevil root[19100] DEBUG switching to TensorFlow for CPU


[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


2020-05-26 15:38:34 darkdevil root[19100] DEBUG Buy at: $10389.55
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $10134.35
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Sell at: $10142.69 | Position: -$246.86
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $10372.25
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $10185.05
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $9721.00
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $9498.44
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $9584.54
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $9587.47
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Sell at: $9724.98 | Position: -$409.37
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Sell at: $10340.00 | Position: -$32.25
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Sell at: $10615.28 | Position: +$430.23
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $10567.02
2020-05-26 15:38:37 darkdevil root[19100] DEBUG Buy at: $105

2020-05-26 15:38:38 darkdevil root[19100] DEBUG Buy at: $8523.61
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Buy at: $8531.88
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Buy at: $8915.24
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Sell at: $8760.07 | Position: -$25.18
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Buy at: $8750.87
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Buy at: $9054.68
2020-05-26 15:38:38 darkdevil root[19100] DEBUG Sell at: $9131.88 | Position: +$438.97
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Buy at: $8886.66
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Buy at: $8033.31
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Buy at: $7929.87
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Buy at: $7894.56
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Buy at: $7934.52
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Sell at: $5028.97 | Position: -$3494.64
2020-05-26 15:38:39 darkdevil root[19100] DEBUG Sell at: $5312.64 | Position: -$3219.24
2

In [69]:
import lxml.html as lh
import time
import urllib.request
import argparse
import urllib.request, urllib.parse, urllib.error
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import json
import ssl
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

In [70]:
tz = pytz.timezone('Asia/Kolkata')
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [71]:
def Real(url):
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    info = json.loads(data)
    live = info['price'];
    time.sleep(1)
    return float(live)

In [72]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=True):
    action = agent.act(state, is_eval=True)
    # BUY
    if action == 1:
        agent.inventory.append(data[t])
        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))
    # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta
        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
    # HOLD
    else:
        history.append((data[t], "HOLD"))

    agent.memory.append((state, action, reward, next_state))
    return total_profit

In [73]:
def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    price = []
    window_size = 10
    time_now = datetime.datetime.now(tz).time()
    while(count < 50):
        url = 'https://api.binance.com/api/v1/ticker/price?symbol={}'.format(args.ticker)
        live = Real(url)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_double-dqn_GOOG_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state

In [74]:
coloredlogs.install(level="DEBUG")
switch_k_backend_device()
args = argparse.Namespace(ticker='BTCUSDT')
try:
    main(args)
except KeyboardInterrupt:
    print("Aborted")  

2020-05-26 15:39:33 darkdevil root[19100] DEBUG switching to TensorFlow for CPU


8862.6


2020-05-26 15:39:46 darkdevil root[19100] DEBUG Buy at: $8856.60
2020-05-26 15:39:46 darkdevil root[19100] INFO model_double-dqn_GOOG_50: USELESS



8860.31


2020-05-26 15:39:49 darkdevil root[19100] DEBUG Buy at: $8857.02
2020-05-26 15:39:49 darkdevil root[19100] INFO model_double-dqn_GOOG_50: USELESS



...

2020-05-26 15:41:19 darkdevil root[19100] DEBUG Buy at: $8852.69
2020-05-26 15:41:19 darkdevil root[19100] INFO model_double-dqn_GOOG_50: USELESS



8853.59


2020-05-26 15:41:21 darkdevil root[19100] DEBUG Buy at: $8852.09
2020-05-26 15:41:21 darkdevil root[19100] INFO model_double-dqn_GOOG_50: USELESS



8855.65


2020-05-26 15:41:24 darkdevil root[19100] DEBUG Buy at: $8855.79
2020-05-26 15:41:24 darkdevil root[19100] INFO model_double-dqn_GOOG_50: USELESS

